# Нейросетевой анализ

Мы приведём два варианта нейросетевого анализа набора данных. Второй заключается в построении свёрточной нейронной сети по изображениям. 

Импортируем инструменты

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Flatten, Input, MaxPooling2D

Импортируем данные

In [ ]:
data = pd.read_parquet("images.parquet")
data.info()
data.head()

Составляем матрицу объекты-признаки и вектор целевой переменной

In [ ]:
X = data.drop(columns=["target"]).values
y = np.array(data["target"].values.reshape(-1, 1), dtype=np.int32)
X.shape, y.shape

Изображения в растянутом виде, необходимо привести их в исходный вид, чтобы использовать свёрточную нейронную сеть. Чтобы грамотно составить набор входных данных, нам необходимо организовать правильный порядок столбцов. 

In [ ]:
%%time

color_names = ["red", "green", "blue"]
columns=[f"{color}_{i}_{j}" for i in range(50) for j in range(50) for color in color_names]

X = data[columns].values
y = np.array(data["target"].values.reshape(-1, 1), dtype=np.int32)
X.shape, y.shape

Разбиваем выборку на обучающую и тестовую 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

Стандартизируем данные

In [ ]:
ss = StandardScaler().fit(X_train)

X_train_scaled = ss.transform(X_train).reshape(-1, 50, 50, 3)
X_test_scaled = ss.transform(X_test).reshape(-1, 50, 50, 3)
X_train_scaled.shape, X_test_scaled.shape

Определяем модель

In [ ]:
model = models.Sequential(name="cnn")

model.add(layers.Conv2D(50, (3, 3), activation='relu', input_shape=(50, 50, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(16, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(16, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))
model.add(Dense(1, activation="sigmoid", name="output_layer", dtype=np.float64))

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

Проводим обучение

In [ ]:
model.fit(X_train_scaled, y_train, epochs=4, batch_size=32)

Проверяем качество

In [ ]:
y_pred = model.predict(X_test_scaled).flatten()

In [ ]:
accuracy = accuracy_score(y_test.flatten(), np.array(y_pred > 0.5, dtype=np.int16))
roc_auc = roc_auc_score(y_test.flatten(), y_pred)
print("accuracy:{:4.6f}\nroc_auc:{:4.6f}".format(accuracy, roc_auc))

**Выводы:** 